In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

if os.path.join(os.getcwd(), '../DeepLearningToolbox') not in sys.path:
    sys.path.append(os.path.join(os.getcwd(), '../DeepLearningToolbox'))
import torch
import datasets
from transformers import GPT2Tokenizer
from torch.utils.data import DataLoader
from utils import dataset_handling

from DeepLearningModules import transformer

# Generate Dataset

In [70]:
# Download the Tiny Shakespeare dataset
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
file_dir ="data"
text_file = os.path.join(file_dir, 'mini_shakespeare.txt')
data_dir = os.path.join(file_dir, 'mini_shakespeare_datasets')

# 90% train, 10% test + validation
dataset_handling.generate_dataset(text_file, data_dir, url=url)

Saving the dataset (1/1 shards): 100%|██████████| 1639/1639 [00:00<00:00, 67877.84 examples/s]


# Load Dataset

In [71]:
dataset = datasets.load_from_disk("data/mini_shakespeare_datasets")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 29499
    })
    valid: Dataset({
        features: ['text'],
        num_rows: 1639
    })
    test: Dataset({
        features: ['text'],
        num_rows: 1639
    })
})


In [85]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
tokenized_dataset = {}
for key in dataset:
    tokenized_dataset[key] = dataset_handling.process_data(dataset[key], tokenizer)

In [93]:
batch_size = 16
dataloaders = {key: DataLoader(tokenized_dataset[key], batch_size=batch_size) for key in tokenized_dataset.keys()}
print(next(iter(dataloaders['train'])))

{'input_ids': tensor([[   40,   481,   351,  ..., 50256, 50256, 50256],
        [   56, 14670,    25,  ..., 50256, 50256, 50256],
        [  464,   886,   286,  ..., 50256, 50256, 50256],
        ...,
        [   46,  4334,  1661,  ..., 50256, 50256, 50256],
        [ 2514,  8616, 10797,  ..., 50256, 50256, 50256],
        [    6, 10915,   477,  ..., 50256, 50256, 50256]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [ ]:
a = transformer.Transformer(embed_size, heads, num_layers, max_length, vocab_length, dropout=0.1, device='cpu')